In [17]:
import pandas as pd
from ibmpairs import paw, authentication
import json
import numpy as np
import logging
import traceback
import os
import glob
import requests

In [18]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.getLogger('ibm_s3transfer').setLevel(logging.INFO)
logging.getLogger('ibm_botocore').setLevel(logging.INFO)
logging.getLogger('ibm_boto3').setLevel(logging.INFO)
logging.getLogger('urllib3').setLevel(logging.INFO)

In [19]:
logging.basicConfig(level=logging.INFO)
pawLogger = logging.getLogger('ibmpairs.paw')
pawLogger.setLevel(logging.INFO)

with open("ibmpairskey.txt", "r") as f:
    pairs_key = f.read()
    
def authenticate(pairs_key):
    
    pairs_credentials = authentication.OAuth2(api_key=pairs_key)
    auth_header = {'Authorization': f'Bearer {pairs_credentials.jwt_token}'}
    PAIRS_SERVER   = 'https://pairs.res.ibm.com'
    PAIRS_CREDENTIALS = authentication.OAuth2(api_key=pairs_key)
    
    return PAIRS_SERVER, PAIRS_CREDENTIALS

#authenticate(pairs_key)

# First, create datalayers

In [32]:
# Get access token

AUTH_PROVIDER_BASE = 'https://auth-b2b-twc.ibm.com'
PAIRS_API_KEY = pairs_key

auth_response = requests.post(    
    AUTH_PROVIDER_BASE + '/connect/token',
    headers={'Content-Type': 'application/x-www-form-urlencoded'},
    data=[
        ('client_id', 'ibm-pairs'),
        ('grant_type', 'apikey'),
        ('apikey', PAIRS_API_KEY)])

auth_obj = auth_response.json()
access_token = auth_obj.get('access_token')
refresh_token = auth_obj.get('refresh_token')
print('access_token: {}'.format(access_token))
print('refresh_token: {}'.format(refresh_token))

access_token: eyJhbGciOiJSUzI1NiIsImtpZCI6IjJENDA5QTlBNjc1MzhBOTU0QUFDRjMyMkU1NjZDNENGOUZENkNBMzIiLCJ0eXAiOiJhdCtqd3QiLCJ4NXQiOiJMVUNhbW1kVGlwVktyUE1pNVdiRXo1X1d5akkifQ.eyJuYmYiOjE2NjcyOTg5MzUsImV4cCI6MTY2NzMwMjUzNSwiaXNzIjoiaHR0cHM6Ly9hdXRoLWIyYi10d2MuaWJtLmNvbSIsImF1ZCI6WyJhY2Nlc3M6YWdybyIsImlibS1wYWlycy1hcGkiLCJwaG9lbml4LWFwaSJdLCJjbGllbnRfaWQiOiJpYm0tcGFpcnMiLCJzdWIiOiJlZmMzYTJjYy1kZTE3LTRlMjktYWE2Yi0zZGU5YzNkM2VlMmUiLCJhdXRoX3RpbWUiOjE2NjcyOTg5MzUsImlkcCI6ImxvY2FsIiwiYXBwbGljYXRpb25zIjoiW1wiQUxFUlRDT05TT0xFXCIsXCJXT0NfSU5URUdSQVRJT05cIixcIklCTVBBSVJTXCIsXCJBR1JPX0FQSVwiLFwiQURNSU5JU1RSQVRJT05cIixcIk9QU0RBU0hCT0FSRFwiLFwiQ0FSQk9OX0FQSVwiLFwiUEFOR0VBXCIsXCJEQVRBSU5URUdSQVRJT05QT1JUQUxcIl0iLCJyb2xlcyI6Ilt7XCJhcHBsaWNhdGlvbklkXCI6XCJBTEVSVENPTlNPTEVcIixcInJvbGVzXCI6W119LHtcImFwcGxpY2F0aW9uSWRcIjpcIkFETUlOSVNUUkFUSU9OXCIsXCJyb2xlc1wiOltcIk5PQUNDRVNTXCJdfSx7XCJhcHBsaWNhdGlvbklkXCI6XCJDQVJCT05fQVBJXCIsXCJyb2xlc1wiOltcIkFETUlOXCJdfSx7XCJhcHBsaWNhdGlvbklkXCI6XCJPUFNEQVNIQk9BUkRcIixcInJvbGV

In [ ]:
import json

with open('layers.json') as data_file:    
    data = json.load(data_file)

count = 0    
for layer in data['layers']:
    json_layer = {
        "layerType": "Raster",
        "layers": [layer]
    }
    # If you want to create one layer at the time and stop at every new layer
    if count==38:
        break
    else:
        count+=1

In [ ]:
json_layer

In [ ]:
dataset_id = '588'
api_response = requests.post(
    'https://pairs.res.ibm.com/v2/datasets/{}/datalayers'.format(dataset_id),
    headers = {
        'Authorization':'Bearer {}'.format(access_token),
        'Content-Type': 'application/json'
    },
    json = json_layer
)

In [ ]:
api_response # 200 id success

In [ ]:
# Creating all the layers at once

for layer in data['layers']:
    json_layer = {
        "layerType": "Raster",
        "layers": [layer]
    }
    dataset_id = '588'
    api_response = requests.post(
        'https://pairs.res.ibm.com/v2/datasets/{}/datalayers'.format(dataset_id),
        headers = {
            'Authorization':'Bearer {}'.format(access_token),
            'Content-Type': 'application/json'
        },
        json = json_layer
    )
    print(api_response)

### Update/correct a layer 

In [33]:
# Give the fields that need to be updated
json_corrected = {
    "description_short": "10m resolution map of the forest composition",
    "description_long": "10m resolution map of the forest composition. Data is between 0 and 100, representing the fraction of broadleaved trees (as opposed to coniferous). NA is no forest. Study restricted to vegetation above 3m."
}

In [34]:
layer_id = '51702'
api_response = requests.put(
    'https://pairs.res.ibm.com/v2/datalayers/{}'.format(layer_id),
    headers = {
        'Authorization':'Bearer {}'.format(access_token),
        'Content-Type': 'application/json'
    },
    json = json_corrected
)
print(api_response)

<Response [200]>


In [ ]:
# Correcting PAIRS level of soil maps

layer_ids = ['51693', '51694', '51695', '51696', '51697', '51698', '51692', '51658', '51659', '51660', '51661', '51662', '51663', 
             '51664', '51665', '51666', '51667', '51668', '51669', '51670', '51671', '51672', '51673', '51674', '51675', '51676',
             '51677', '51682' , '51683', '51684', '51685', '51686', '51687', '51688', '51689', '51690', '51691']

json_corrected = {
    "level": "22"
}

for layer_id in layer_ids:
    api_response = requests.put(
        'https://pairs.res.ibm.com/v2/datalayers/{}'.format(layer_id),
        headers = {
            'Authorization':'Bearer {}'.format(access_token),
            'Content-Type': 'application/json'
        },
        json = json_corrected
    )
    print(api_response)

In [ ]:
# Correcting upload interpolation method

layer_ids = ['51693', '51694', '51695', '51696', '51697', '51698', '51692', '51658', '51659', '51660', '51661', '51662', '51663', 
             '51664', '51665', '51666', '51667', '51668', '51669', '51670', '51671', '51672', '51673', '51674', '51675', '51676',
             '51677', '51682' , '51683', '51684', '51685', '51686', '51687', '51688', '51689', '51690', '51691']

json_corrected = {
    "upload_interpolation": "bilinear"
}

for layer_id in layer_ids:
    api_response = requests.put(
        'https://pairs.res.ibm.com/v2/datalayers/{}'.format(layer_id),
        headers = {
            'Authorization':'Bearer {}'.format(access_token),
            'Content-Type': 'application/json'
        },
        json = json_corrected
    )
    print(api_response)

### Increase number of layers possible in dataset

In [ ]:
new_max = {
    "maxLayers" : 55
}

In [ ]:
dataset_id = '588'
api_response = requests.put(
    'https://pairs.res.ibm.com/v2/datasets/{}'.format(dataset_id),
    headers = {
        'Authorization':'Bearer {}'.format(access_token),
        'Content-Type': 'application/json'
    },
    json = new_max
)
print(api_response)

# Uploading to IBM PAIRS

This notebook illustrates how to use the `uploads` module to upload to IBM PAIRS via IBM Cloud Object Store (COS).

In [20]:
#from ibmpairs import uploads
import uploads_key_auth as uploads #local script
import logging
from pathlib import Path

We want full debug logging for the uploads module.

In [21]:
#logging.basicConfig(level=logging.DEBUG)

However, we want these other modules to be quiet.

In [22]:
logging.getLogger('ibm_s3transfer').setLevel(logging.INFO)
logging.getLogger('ibm_botocore').setLevel(logging.INFO)
logging.getLogger('ibm_boto3').setLevel(logging.INFO)
logging.getLogger('urllib3').setLevel(logging.INFO)

## Credentials

Not surprisingly, we need a whole host of credentials. The first 7 of these are COS related, the final is just PAIRS username&password.

In [23]:
apikey = "FzmaPxVOTb6_LOH1AqNggpFwER7n61EISx1_TqrQugKU"
resourceInstanceID = "crn:v1:bluemix:public:cloud-object-storage:global:a/045077ab9f1a4dcfafa2c58389d3d639:11a6c883-43a0-4758-a264-6568f2b5ad9a::"
authEndpoint = "https://iam.eu-de.bluemix.net/oidc/token"
endpoint = "https://s3.eu-de.cloud-object-storage.appdomain.cloud"
bucketName = "drought"
accessKeyID = "b9f5af1d0bae45e4bbd9a58420a6190c"
secretAccessKey = "53faf26559acb6e0752d16d869cabff2016a1991b97d216a"

In [24]:
import os, json, logging, sys, traceback, asyncio, aiohttp, warnings

# Get access token

AUTH_PROVIDER_BASE = 'https://auth-b2b-twc.ibm.com'
PAIRS_API_KEY = pairs_key

auth_response = requests.post(    
    AUTH_PROVIDER_BASE + '/connect/token',
    headers={'Content-Type': 'application/x-www-form-urlencoded'},
    data=[
        ('client_id', 'ibm-pairs'),
        ('grant_type', 'apikey'),
        ('apikey', PAIRS_API_KEY)])

auth_obj = auth_response.json()
access_token = auth_obj.get('access_token')
refresh_token = auth_obj.get('refresh_token')

pairsAuthHeader = {
        'Authorization':'Bearer {}'.format(access_token),
        'Content-Type': 'application/json'
    }

## Uploading

### Create .meta.json files for each file to upload

Need to create a `.meta.json` file for each data file

In [ ]:
path_file = '/Users/led/Desktop/Master_Thesis/uploads to PAIRS.xlsx'
data_dir = '/Users/led/Desktop/Master_Thesis/Data/new_layers/'
file_to_id = pd.read_excel(path_file)

In [ ]:
for idx, row in file_to_id.iterrows():
    meta_json = {
        "datalayer_id": [str(int(row["PAIRS ID"]))],
        "pairsdatatype": "2draster",
        "datainterpolation": "bilinear",
        "timestamp": str(row["Timestamp"]),
    }
    
    # Serializing json
    json_object = json.dumps(meta_json, indent=4)

    # Writing to sample.json
    file_prefix = row["Processed data (uploaded to PAIRS)"]
    with open(data_dir + f"{file_prefix}.meta.json", "w") as outfile:
        outfile.write(json_object)

### Uploading local data via IBM COS

To start, we initialize a `CosBucket` object. This simply soaks up all those pesky credentials we had to define above.

In [ ]:
cosBucket = uploads.IBMCosBucket(
    apikey, resourceInstanceID, authEndpoint, endpoint,
    bucketName, accessKeyID, secretAccessKey
)

For now, we want to upload 3 files from our local disk. For each of these, there is a `.meta.json` file in the same location, which contains meta information in **JSON** format. (I.e. the format accepted by the web uploader.)

In [25]:
filesToUpload = [
    '/Users/led/Desktop/Master_Thesis/Data/new_layers/Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff'
]

"""
'/Users/led/Desktop/Master_Thesis/Data/new_layers/Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/Waldmischungsgrad_2018_10m_2056_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-0_5-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-5_15-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-15_30-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-30_60-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-60_100-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-100_200-rf_reprojected.tiff'
'/Users/led/Desktop/Master_Thesis/Data/new_layers/mittlere-gruendigkeit-p-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ph-cacl2-0_5-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ph-cacl2-5_15-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ph-cacl2-15_30-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ph-cacl2-30_60-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ph-cacl2-60_100-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ph-cacl2-100_200-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/sand-final-0_5-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/sand-final-5_15-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/sand-final-15_30-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/sand-final-30_60-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/sand-final-60_100-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/sand-final-100_200-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/skelettgehalt-0_5-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/skelettgehalt-5_15-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/skelettgehalt-15_30-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/skelettgehalt-30_60-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/skelettgehalt-60_100-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/skelettgehalt-100_200-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ton-final-0_5-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ton-final-5_15-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ton-final-15_30-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ton-final-30_60-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ton-final-60_100-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/ton-final-100_200-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/trd-feinerde-final-0_5-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/trd-feinerde-final-5_15-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/trd-feinerde-final-15_30-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/trd-feinerde-final-30_60-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/trd-feinerde-final-60_100-rf_reprojected.tiff',
'/Users/led/Desktop/Master_Thesis/Data/new_layers/trd-feinerde-final-100_200-rf_reprojected.tiff'
"""

"\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff',\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/Waldmischungsgrad_2018_10m_2056_reprojected.tiff',\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-0_5-rf_reprojected.tiff',\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-5_15-rf_reprojected.tiff',\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-15_30-rf_reprojected.tiff',\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-30_60-rf_reprojected.tiff',\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-60_100-rf_reprojected.tiff',\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/corg-gehalt-final-100_200-rf_reprojected.tiff'\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/mittlere-gruendigkeit-p-rf_reprojected.tiff',\n'/Users/led/Desktop/Master_Thesis/Data/new_layers/ph-cacl2-0_5-rf_reprojected.tiff',\n'/Users/le

We finally create a `PAIRSUpload` object, that will upload our 3 files.

In [26]:
uploadFromLocal = uploads.PAIRSUpload.fromLocal(
    cosBucket, pairsAuthHeader, False, fileList=filesToUpload
)

INFO:uploads_key_auth:'PAIRSUpload'. Calling 'run' will upload 1 files. Total raw data volume 2809128206 bytes. Depending on the PAIRS resolution level of target layer(s), the data volume might be significantly different in IBM PAIRS.


We will now run the actual upload. `uploads` uses asynchronous python calls. The benefit is that we get a level of parallelization without using threads. In the following, we use 3 workers. Because this is a coroutine, we have to *await* it.

In [27]:
await uploadFromLocal.run(1)

DEBUG:uploads_key_auth:Commencing upload run.
DEBUG:uploads_key_auth:Initialized ClientSession.
DEBUG:uploads_key_auth:Worker picked up /Users/led/Desktop/Master_Thesis/Data/new_layers/Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff.
DEBUG:uploads_key_auth:Pushing Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff to COS.
DEBUG:uploads_key_auth:Pushing Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff.meta.json to COS.
DEBUG:uploads_key_auth:Worker moved Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff to COS.
DEBUG:uploads_key_auth:Worker will obtain meta data for Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff.
DEBUG:uploads_key_auth:Worker has obtained meta data for Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff.
DEBUG:uploads_key_auth:Starting PAIRS upload.
DEBUG:uploads_key_auth:Submitting upload request to PAIRS.
DEBUG:uploads_key_auth:Upload request submitted to PAI

We check if any uploads have not succeeded.

In [30]:
{v.remoteObject for v in uploadFromLocal.completedUploads if not v.status == 'SUCCEEDED'}

set()

In [31]:
uploadFromLocal.completedUploads

[Status = SUCCEEDED localFile = /Users/led/Desktop/Master_Thesis/Data/new_layers/Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff remoteObject = Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff size = 2809128206 uploadInfo = {'tracking_id': 'iqyjwfzbsgsnmrlcdafvnltpovravyvszlogmupq', 'progress': 100, 'status': 'SUCCEEDED', 'summary': [{'raw_filename': 'qszuvfmnme-mvxajmjezw-1667297494-iqyjwfzbsgsnmrlcdafvnltpovravyvszlogmupq', 'status': 4.0, 'details': 'Finished: Succeeded ingesting file into PAIRS', 'last_updated': '2022-11-01T06:32:08.200536-04:00'}], 'user_tag': '', 'last_updated': '2022-11-01 06:32:08.205069-04:00'} exception = None]

In [ ]:
#cosBucket.delete(filename)

### Uploading directly from COS

We will now upload the same data *again*. However, since the data is already in COS, we will initialize our `PAIRSUplaod` object from COS. This illustrates the alternate use of `PAIRSUpload`. Moreover, we will remove the data from COS afterwards.

In [9]:
cosBucket = uploads.IBMCosBucket(
    apikey, resourceInstanceID, authEndpoint, endpoint,
    bucketName, accessKeyID, secretAccessKey
)

In [10]:
cosBucket.updateCosObjects(Prefix='V', MaxKeys=4096)
cosObjects = cosBucket.cosObjects()

INFO:uploads_key_auth:IBMCosBucket with 1 uploadable objects, 2809128206 bytes total.


In [11]:
uploadFromCos = uploads.PAIRSUpload.fromCos(
    cosBucket, pairsAuthHeader, True, Prefix='V', MaxKeys=4096
)

INFO:uploads_key_auth:IBMCosBucket with 1 uploadable objects, 2809128206 bytes total.
INFO:uploads_key_auth:'PAIRSUpload'. Calling 'run' will upload 1 files. Total raw data volume 2809128206 bytes. Depending on the PAIRS resolution level of target layer(s), the data volume might be significantly different in IBM PAIRS.


In [12]:
len(uploadFromCos.pendingUploads)

1

Once again, we have to await the actual upload. We use one worker. (If you are fancy, you can wrap this with asyncio.create_task and continue doing other stuff in jupyter while this is running.)

In [13]:
await uploadFromCos.run(nWorkers=1)

DEBUG:uploads_key_auth:Commencing upload run.
DEBUG:uploads_key_auth:Initialized ClientSession.
DEBUG:uploads_key_auth:Worker picked up None.
DEBUG:uploads_key_auth:Worker will obtain meta data for Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff.
DEBUG:uploads_key_auth:Worker has obtained meta data for Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff.
DEBUG:uploads_key_auth:Starting PAIRS upload.
DEBUG:uploads_key_auth:Submitting upload request to PAIRS.
DEBUG:uploads_key_auth:Upload request submitted to PAIRS. Status 201.
DEBUG:uploads_key_auth:Upload request submitted. JSON response decoded.
DEBUG:uploads_key_auth:Registered PAIRS upload aybvteuvhscmuzfdiancjmgqmrmvapilkmmcrjqc.
DEBUG:uploads_key_auth:Checking PAIRS upload status aybvteuvhscmuzfdiancjmgqmrmvapilkmmcrjqc.
DEBUG:uploads_key_auth:Checking PAIRS upload status aybvteuvhscmuzfdiancjmgqmrmvapilkmmcrjqc.
DEBUG:uploads_key_auth:Checking PAIRS upload status aybvteuvhscmuzfdiancjmgqmrmvapi

Once again, we can verify that all uploads completed running.

In [14]:
{v.remoteObject for v in uploadFromCos.completedUploads if not v.status == 'SUCCEEDED'}

{'Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff'}

In [15]:
uploadFromCos.completedUploads

[Status = FAILED localFile = None remoteObject = Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff size = 2809128206 uploadInfo = {'tracking_id': 'aybvteuvhscmuzfdiancjmgqmrmvapilkmmcrjqc', 'progress': 15, 'status': 'RUNNING', 'summary': [{'raw_filename': '1667296107-aybvteuvhscmuzfdiancjmgqmrmvapilkmmcrjqc', 'status': 0.59, 'details': 'File finished downloading from URL.', 'last_updated': '2022-11-01 05:54:19.467038-04:00'}, {'raw_filename': 'zcjxikudvs-ifkpmtlpay-1667296107-aybvteuvhscmuzfdiancjmgqmrmvapilkmmcrjqc', 'status': -3.0, 'details': 'Failed ingesting the file into PAIRS: General Error: ', 'last_updated': '2022-11-01 06:00:40.115877-04:00'}], 'user_tag': '', 'last_updated': '2022-11-01 05:48:27.206451-04:00'} exception = None]

In [16]:
cosBucket.delete('Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff')

DEBUG:uploads_key_auth:Deleting Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff and Vegetationshoehenmodell_2021_10m_LFI_ID164_19_LV95_reprojected.tiff.meta.json.
